In [1]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np # Necesitaremos numpy para algunos cálculos

# Configuración para que pandas muestre todas las columnas
pd.set_option('display.max_columns', None)

print("Librerías importadas.")

Librerías importadas.


In [2]:
# Definimos la ruta al archivo procesado de la Fase 1
RUTA_DATOS_PROCESADOS = os.path.join('..', '1_datos', '02_procesados')
RUTA_GPKG_ENTRADA = os.path.join(RUTA_DATOS_PROCESADOS, 'gdf_maestro_manzanillo.gpkg')

# Cargamos el GeoDataFrame
try:
    gdf = gpd.read_file(RUTA_GPKG_ENTRADA)
    print("GeoDataFrame maestro cargado exitosamente.")
    print(f"Dimensiones de la tabla: {gdf.shape}")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    print("Asegúrate de haber ejecutado el notebook 1 y guardado el archivo .gpkg correctamente.")

# Mostramos las primeras filas para recordar qué contiene
display(gdf.head())

GeoDataFrame maestro cargado exitosamente.
Dimensiones de la tabla: (65, 238)


,ID,ENTIDAD,DISTRITO_F,DISTRITO_L,MUNICIPIO,seccion,TIPO,CONTROL,GEOMETRY1_,lista_nominal_promedio,votos_totales_acumulados,votos_morena_acumulados,votos_oposicion_acumulados,partido_dominante,pct_voto_morena,tasa_participacion_promedio,DISTRITO,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_0A17,P_3YMAS,P_3YMAS_F,P_3YMAS_M,P_5YMAS,P_5YMAS_F,P_5YMAS_M,P_12YMAS,P_12YMAS_F,P_12YMAS_M,P_15YMAS,P_15YMAS_F,P_15YMAS_M,P_18YMAS,P_18YMAS_F,P_18YMAS_M,P_3A5,P_3A5_F,P_3A5_M,P_6A11,P_6A11_F,P_6A11_M,P_8A14,P_8A14_F,P_8A14_M,P_12A14,P_12A14_F,P_12A14_M,P_15A17,P_15A17_F,P_15A17_M,P_18A24,P_18A24_F,P_18A24_M,P_15A49_F,P_60YMAS,P_60YMAS_F,P_60YMAS_M,REL_H_M,POB0_14,POB15_64,POB65_MAS,POB_EDADNE,PROM_HNV,PNACENT,PNACENT_F,PNACENT_M,PNACOE,PNACOE_F,PNACOE_M,PRES2015,PRES2015_F,PRES2015_M,PRESOE15,PRESOE15_F,PRESOE15_M,P3YM_HLI,P3YM_HLI_F,P3YM_HLI_M,P3HLINHE,P3HLINHE_F,P3HLINHE_M,P3HLI_HE,P3HLI_HE_F,P3HLI_HE_M,P5_HLI,P5_HLI_NHE,P5_HLI_HE,PHOG_IND,POB_AFRO,POB_AFRO_F,POB_AFRO_M,PCON_DISC,PCDISC_MOT,PCDISC_VIS,PCDISC_LEN,PCDISC_AUD,PCDISC_M_A,PCDISC_MEN,PCON_LIMI,PCLIM_CSB,PCLIM_VIS,PCLIM_HACO,PCLIM_OAUD,PCLIM_MOT2,PCLIM_RE_C,PCLIM_PMEN,PSIND_LIM,P3A5_NOA,P3A5_NOA_F,P3A5_NOA_M,P6A11_NOA,P6A11_NOAF,P6A11_NOAM,P12A14NOA,P12A14NOAF,P12A14NOAM,P15A17A,P15A17A_F,P15A17A_M,P18A24A,P18A24A_F,P18A24A_M,P8A14AN,P8A14AN_F,P8A14AN_M,P15YM_AN,P15YM_AN_F,P15YM_AN_M,P15YM_SE,P15YM_SE_F,P15YM_SE_M,P15PRI_IN,P15PRI_INF,P15PRI_INM,P15PRI_CO,P15PRI_COF,P15PRI_COM,P15SEC_IN,P15SEC_INF,P15SEC_INM,P15SEC_CO,P15SEC_COF,P15SEC_COM,P18YM_PB,P18YM_PB_F,P18YM_PB_M,GRAPROES,GRAPROES_F,GRAPROES_M,PEA,PEA_F,PEA_M,PE_INAC,PE_INAC_F,PE_INAC_M,POCUPADA,POCUPADA_F,POCUPADA_M,PDESOCUP,PDESOCUP_F,PDESOCUP_M,PSINDER,PDER_SS,PDER_IMSS,PDER_ISTE,PDER_ISTEE,PAFIL_PDOM,PDER_SEGP,PDER_IMSSB,PAFIL_IPRI,PAFIL_OTRA,P12YM_SOLT,P12YM_CASA,P12YM_SEPA,TOTHOG,HOGJEF_F,HOGJEF_M,POBHOG,PHOGJEF_F,PHOGJEF_M,VIVTOT,TVIVHAB,TVIVPAR,VIVPAR_HAB,VIVPARH_CV,TVIVPARHAB,VIVPAR_DES,VIVPAR_UT,OCUPVIVPAR,PROM_OCUP,PRO_OCUP_C,VPH_PISODT,VPH_PISOTI,VPH_1DOR,VPH_2YMASD,VPH_1CUART,VPH_2CUART,VPH_3YMASC,VPH_C_ELEC,VPH_S_ELEC,VPH_AGUADV,VPH_AEASP,VPH_AGUAFV,VPH_TINACO,VPH_CISTER,VPH_EXCSA,VPH_LETR,VPH_DRENAJ,VPH_NODREN,VPH_C_SERV,VPH_NDEAED,VPH_DSADMA,VPH_NDACMM,VPH_SNBIEN,VPH_REFRI,VPH_LAVAD,VPH_HMICRO,VPH_AUTOM,VPH_MOTO,VPH_BICI,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCIN,VPH_SINTIC,geometry
0,243.0,6,2,12,8,254,2,209.0,None,478.458333,8258,1728,6297,pan,21.530027,71.908743,2,660,339,321,19,5,14,122,582,306,276,571,299,272,529,276,253,495,261,234,479,254,225,16,11,5,37,19,18,60,32,28,34,15,19,16,7,9,54,27,27,143,132,72,60,94.69,106,406,89,59,1.57,318,162,156,270,140,130,502,257,245,69,42,27,0,0,0,0,0,0,0,0,0,0,0,0,0,8,1,7,15,9,5,1,2,3,2,44,11,30,1,7,1,2,5,539,6,4,2,2,2,0,4,2,2,11,6,5,26,13,13,1,0,1,3,2,1,3,2,1,11,5,6,23,15,8,9,5,4,63,31,32,376,198,178,13.27,13.07,13.50,304,127,177,224,148,76,302,127,175,2,0,2,100,501,321,37,2,91,31,0,23,1,194,253,82,244,80,164,659,167,492,333,245,280,192,244,244,56,32,659,2.70,0.80,219,3,76,146,14,55,153,221,1,222,222,0,186,3,222,0,222,0,221,0,222,75,1,214,185,117,140,22,78,154,209,148,146,213,183,141,78,30,12,5,25,2,"MULTIPOLYGON (((573667.542 2109247.202, 573680..."
1,244.0,6,2,12,8,255,2,210.0,None,312.541667,5749,1645,3938,morena,29.459169,76.632898,2,375,200,175,4,1,3,70,369,198,171,359,195,164,334,183,151,320,175,145,303,167,136,14,5,9,21,10,11,30,15,15,14,8,6,17,8,9,44,21,23,87,88,46,42,87.50,53,255,65,2,1.79,223,123,100,144,74,70,353,192,161,5,3,2,1,1,0,0,0,0,1,1,0,1,0,1,2,0,0,0,11,6,3,3,4,1,0,35,14,20,0,4,0,5,4,325,6,2,4,4,3,1,0,0,0,13,8,5,15,7,8,1,1,0,4,3,1,4,3,1,13,7,6,39,24,15,7,2,5,47,25,22,198,108,90,11.62,11.48,11.80,184,81,103,147,99,48,178,80,98,6,1,5,73,300,199,21,1,61,21,0,3,0,143,149,42,132,49,83,375,120,255,207,132,183,108,132,132,53,22,375,2.84,0.79,131,0,33,98,3,22,106,131,0,131,131,0,101,5,131,0,131,0,131,0,131,57,2,124,119,42,69,13,34,79,123,71,78,116,91,63,34,14,8,11,33,5,"MULTIPOLYGON (

In [3]:
# --- Celda 3 Definitiva: Ingeniería de Atributos Demográficos ---

print("Creando atributos demográficos a partir de tu selección de variables...")

COL_TOTAL_POB = 'POBTOT'
COL_TOTAL_VIV = 'VIVPAR_HAB' # Total de Viviendas Particulares Habitadas
COL_TOTAL_HOG = 'TOTHOG'     # Total de Hogares
COL_PEA = 'PEA'          # Población Económicamente Activa

# --- PILAR 1: EDAD Y CICLO DE VIDA ---
if COL_TOTAL_POB in gdf.columns and gdf[COL_TOTAL_POB].sum() > 0:
    print("\nCalculando atributos de Edad y Ciclo de Vida...")
    gdf['porc_genero_fem'] = (gdf['POBFEM'] / gdf[COL_TOTAL_POB]) * 100
    gdf['porc_ninos'] = ((gdf.get('P_3A5', 0) + gdf.get('P_6A11', 0)) / gdf[COL_TOTAL_POB]) * 100
    gdf['porc_futuros_votantes'] = (gdf.get('P_15A17', 0) / gdf[COL_TOTAL_POB]) * 100
    gdf['porc_jovenes'] = (gdf.get('P_18A24', 0) / gdf[COL_TOTAL_POB]) * 100
    gdf['porc_adultos_mayores'] = (gdf.get('POB65_MAS', 0) / gdf[COL_TOTAL_POB]) * 100

# --- PILAR 2: NIVEL SOCIOECONÓMICO ---
if COL_TOTAL_VIV in gdf.columns and gdf[COL_TOTAL_VIV].sum() > 0:
    print("\nCalculando atributos de Nivel Socioeconómico...")
    gdf['porc_piso_tierra'] = (gdf.get('VPH_PISOTI', 0) / gdf[COL_TOTAL_VIV]) * 100
    gdf['porc_con_auto'] = (gdf.get('VPH_AUTOM', 0) / gdf[COL_TOTAL_VIV]) * 100
    gdf['porc_con_pc'] = (gdf.get('VPH_PC', 0) / gdf[COL_TOTAL_VIV]) * 100
    gdf['porc_con_celular'] = (gdf.get('VPH_CEL', 0) / gdf[COL_TOTAL_VIV]) * 100
    gdf['porc_con_internet'] = (gdf.get('VPH_INTER', 0) / gdf[COL_TOTAL_VIV]) * 100
    
    # Creación de un índice compuesto de digitalización
    gdf['indice_digitalizacion'] = gdf[['porc_con_pc', 'porc_con_celular', 'porc_con_internet']].mean(axis=1)

# --- PILAR 3: ESTRUCTURA FAMILIAR Y ARRAIGO ---
if COL_TOTAL_HOG in gdf.columns and gdf[COL_TOTAL_HOG].sum() > 0:
     print("\nCalculando atributos de Estructura Familiar y Arraigo...")
     gdf['porc_hogares_jefa_mujer'] = (gdf.get('HOGJEF_F', 0) / gdf[COL_TOTAL_HOG]) * 100
if COL_TOTAL_POB in gdf.columns and gdf[COL_TOTAL_POB].sum() > 0:
    gdf['porc_poblacion_migrante'] = (gdf.get('PNACOE', 0) / gdf[COL_TOTAL_POB]) * 100

# --- PILAR 4: OCUPACIÓN Y BIENESTAR ---
if COL_PEA in gdf.columns and gdf[COL_PEA].sum() > 0:
    print("\nCalculando atributos de Ocupación y Bienestar...")
    gdf['tasa_desocupacion'] = (gdf.get('PDESOCUP', 0) / gdf[COL_PEA]) * 100
if COL_TOTAL_POB in gdf.columns and gdf[COL_TOTAL_POB].sum() > 0:
    gdf['porc_sin_servicios_salud'] = (gdf.get('PSINDER', 0) / gdf[COL_TOTAL_POB]) * 100


# --- Verificación Final de Atributos Creados ---
print("\n--- Vista previa de algunos de los nuevos atributos clave ---")
columnas_nuevas_a_mostrar = [
    'seccion', 'porc_jovenes', 'porc_adultos_mayores', 'GRAPROES', 'indice_digitalizacion',
    'porc_hogares_jefa_mujer', 'porc_poblacion_migrante', 'tasa_desocupacion', 'porc_sin_servicios_salud'
]
columnas_existentes = [col for col in columnas_nuevas_a_mostrar if col in gdf.columns]
display(gdf[columnas_existentes].head())

Creando atributos demográficos a partir de tu selección de variables...

Calculando atributos de Edad y Ciclo de Vida...

Calculando atributos de Nivel Socioeconómico...

Calculando atributos de Estructura Familiar y Arraigo...

Calculando atributos de Ocupación y Bienestar...

--- Vista previa de algunos de los nuevos atributos clave ---


,seccion,porc_jovenes,porc_adultos_mayores,GRAPROES,indice_digitalizacion,porc_hogares_jefa_mujer,porc_poblacion_migrante,tasa_desocupacion,porc_sin_servicios_salud
0,254,8.181818,13.484848,13.27,94.444444,32.786885,40.909091,0.657895,15.151515
1,255,11.733333,17.333333,11.62,85.802469,37.121212,38.400000,3.260870,19.466667
2,201,0.000000,42.857143,11.17,75.000000,75.000000,57.142857,0.000000,0.000000
3,202,10.852713,11.369509,9.18,62.381664,39.972715,30.404823,1.298701,12.790698
4,200,11.111111,9.339775,10.08,76.482618,25.133690,38.969404,2.960526,11.594203


In [4]:
print("Creando atributos electorales...")

# Porcentaje de voto de la oposición (basado en tus columnas agregadas)
gdf['pct_voto_oposicion'] = (gdf['votos_oposicion_acumulados'] / gdf['votos_totales_acumulados']) * 100
print("- Atributo creado: 'pct_voto_oposicion'")

# Nivel de competitividad: qué tan cerrada es la elección en esa sección.
# Un valor cercano a 0 significa muy competido. Un valor cercano a 100 significa dominancia total de un bando.
gdf['competitividad'] = abs(gdf['pct_voto_morena'] - gdf['pct_voto_oposicion'])
print("- Atributo creado: 'competitividad'")

# Verificamos las nuevas columnas
print("\nVista previa de los nuevos atributos electorales:")
display(gdf[['seccion', 'pct_voto_morena', 'pct_voto_oposicion', 'competitividad', 'tasa_participacion_promedio']].head())

Creando atributos electorales...
- Atributo creado: 'pct_voto_oposicion'
- Atributo creado: 'competitividad'

Vista previa de los nuevos atributos electorales:


,seccion,pct_voto_morena,pct_voto_oposicion,competitividad,tasa_participacion_promedio
0,254,21.530027,76.253330,54.723303,71.908743
1,255,29.459169,68.498869,39.039700,76.632898
2,201,17.012780,81.030445,64.017665,90.021082
3,202,34.044820,63.552914,29.508094,66.133679
4,200,31.726496,66.548161,34.821666,67.630926


In [5]:
print("--- Información del GeoDataFrame Enriquecido ---")
gdf.info()

# Reemplazamos cualquier posible valor infinito (inf) o no numérico (NaN) que se haya podido generar
# en las divisiones con el valor 0. Es una buena práctica de limpieza.
gdf.replace([np.inf, -np.inf], np.nan, inplace=True)
gdf.fillna(0, inplace=True)

print("\nLimpieza de valores infinitos o nulos completada.")

--- Información del GeoDataFrame Enriquecido ---
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 65 entries, 0 to 64
Columns: 255 entries, ID to competitividad
dtypes: float64(29), geometry(1), int64(223), object(2)
memory usage: 129.6+ KB

Limpieza de valores infinitos o nulos completada.


/var/folders/81/fmf0cf1s1914f07ysr6xlv100000gn/T/ipykernel_69825/3832856231.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf.fillna(0, inplace=True)


In [6]:
# Guardamos el archivo enriquecido en la carpeta de procesados.
RUTA_SALIDA_ENRIQUECIDA = os.path.join(RUTA_DATOS_PROCESADOS, 'gdf_maestro_enriquecido.gpkg')

try:
    gdf.to_file(RUTA_SALIDA_ENRIQUECIDA, driver='GPKG')
    print(f"¡Éxito! GeoDataFrame enriquecido guardado en:")
    print(RUTA_SALIDA_ENRIQUECIDA)
except Exception as e:
    print(f"Ocurrió un error al guardar el archivo: {e}")

¡Éxito! GeoDataFrame enriquecido guardado en:
../1_datos/02_procesados/gdf_maestro_enriquecido.gpkg
